In [1]:
import os
from os.path import isfile, join
import numpy as np
from PIL import Image, ImageOps
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sympy.geometry import Point, Circle
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive
from IPython.display import clear_output, display
from tkinter import Tk, filedialog
import imageio

%matplotlib inline

In [2]:
#!/usr/bin/env python3.6
# -*- Coding: UTF-8 -*-
"""
Defisheye algorithm.
Developed by: E. S. Pereira.
e-mail: pereira.somoza@gmail.com
Based in the work of F. Weinhaus.
http://www.fmwconcepts.com/imagemagick/defisheye/index.php
Copyright [2019] [E. S. Pereira]
   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at
       http://www.apache.org/licenses/LICENSE-2.0
   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
"""
import cv2
from numpy import arange, sqrt, arctan, sin, tan, zeros, array, meshgrid, pi, ndarray
from numpy import argwhere, hypot


class Defisheye:
    """
    Defisheye
    fov: fisheye field of view (aperture) in degrees
    pfov: perspective field of view (aperture) in degrees
    xcenter: x center of fisheye area
    ycenter: y center of fisheye area
    radius: radius of fisheye area
    angle: image rotation in degrees clockwise
    dtype: linear, equalarea, orthographic, stereographic
    format: circular, fullframe
    """

    def __init__(self, infile, **kwargs):
        vkwargs = {"fov": 180,
                   "pfov": 120,
                   "xcenter": None,
                   "ycenter": None,
                   "radius": None,
                   "angle": 0,
                   "dtype": "equalarea",
                   "format": "fullframe"
                   }
        self._start_att(vkwargs, kwargs)

        if type(infile) == str:
            _image = cv2.imread(infile)
        elif type(infile) == ndarray:
            _image = infile
        else:
            raise ImageError("Image format not recognized")


        width = _image.shape[1]
        height = _image.shape[0]
        xcenter = width // 2
        ycenter = height  // 2

        dim = min(width, height)
        x0 = xcenter - dim // 2
        xf = xcenter + dim // 2
        y0 = ycenter - dim // 2
        yf = ycenter + dim // 2

        self._image = _image[y0:yf, x0:xf, :]

        self._width = self._image.shape[1]
        self._height = self._image.shape[0]

        if self._xcenter is None:
            self._xcenter = (self._width - 1) // 2

        if self._ycenter is None:
            self._ycenter = (self._height - 1) // 2

    def _map(self, i, j, ofocinv, dim):

        xd = i - self._xcenter
        yd = j - self._ycenter

        rd = hypot(xd, yd)
        phiang = arctan(ofocinv * rd)

        if self._dtype == "linear":
            ifoc = dim * 180 / (self._fov * pi)
            rr = ifoc * phiang
            # rr = "rr={}*phiang;".format(ifoc)

        elif self._dtype == "equalarea":
            ifoc = dim / (2.0 * sin(self._fov * pi / 720))
            rr = ifoc * sin(phiang / 2)
            # rr = "rr={}*sin(phiang/2);".format(ifoc)

        elif self._dtype == "orthographic":
            ifoc = dim / (2.0 * sin(self._fov * pi / 360))
            rr = ifoc * sin(phiang)
            # rr="rr={}*sin(phiang);".format(ifoc)

        elif self._dtype == "stereographic":
            ifoc = dim / (2.0 * tan(self._fov * pi / 720))
            rr = ifoc * tan(phiang / 2)

        rdmask = rd != 0
        xs = xd.copy()
        ys = yd.copy()

        xs[rdmask] = (rr[rdmask] / rd[rdmask]) * xd[rdmask] + self._xcenter
        ys[rdmask] = (rr[rdmask] / rd[rdmask]) * yd[rdmask] + self._ycenter

        xs[~rdmask] = 0
        ys[~rdmask] = 0

        xs = xs.astype(int)
        ys = ys.astype(int)
        return xs, ys

    def convert(self, outfile):
        if self._format == "circular":
            dim = min(self._width, self._height)
        elif self._format == "fullframe":
            dim = sqrt(self._width ** 2.0 + self._height ** 2.0)

        if self._radius is not None:
            dim = 2 * self._radius

        # compute output (perspective) focal length and its inverse from ofov
        # phi=fov/2; r=N/2
        # r/f=tan(phi);
        # f=r/tan(phi);
        # f= (N/2)/tan((fov/2)*(pi/180)) = N/(2*tan(fov*pi/360))

        ofoc = dim / (2 * tan(self._pfov * pi / 360))
        ofocinv = 1.0 / ofoc

        i = arange(self._width)
        j = arange(self._height)
        i, j = meshgrid(i, j)

        xs, ys, = self._map(i, j, ofocinv, dim)
        img = self._image.copy()

        img[i, j, :] = self._image[xs, ys, :]
        return img

    def _start_att(self, vkwargs, kwargs):
        """
        Starting atributes
        """
        pin = []

        for key, value in kwargs.items():
            if key not in vkwargs:
                raise NameError("Invalid key {}".format(key))
            else:
                pin.append(key)
                setattr(self, "_{}".format(key), value)

        pin = set(pin)
        rkeys = set(vkwargs.keys()) - pin
        for key in rkeys:
            setattr(self, "_{}".format(key), vkwargs[key])

In [3]:
class main:
    def __init__(self, path):
        self.path = path
        self.defish_dtype = 'linear'
        self.defish_format = 'fullframe'
        self.defish_fov = 180
        self.defish_pfov = 120
        l_files = [files for files in os.listdir(self.path) if files.endswith(".mp4") or files.endswith(".avi")]
        l_videos = [video(filename, self.path) for filename in l_files]
        circle_gui(l_videos)

In [9]:
class video(main):
    def __init__(self, filename, path):
        self.filename = filename
        self.video_path = path + filename
        video = imageio.get_reader(self.video_path,  'ffmpeg')
        self.fps = video.get_meta_data()["fps"]
        self.size = video.get_meta_data()["size"]
        self.frame_number = video.count_frames()
        self.defish_settings = {}
        self.video_output_path = path + filename[0:-4] + "_defisheyed.mp4"
        self.frame = video.get_data(0)
        
    def f(self, Point1_x, Point1_y, Point2_x, Point2_y, Point3_x, Point3_y):
        three_points = [[Point1_x, Point1_y], [Point2_x, Point2_y], [Point3_x, Point3_y]]
        fig = plt.figure(figsize=(18, 10))
        gs = fig.add_gridspec(3, 5)

        fig1 = fig.add_subplot(gs[0:2, 0:2])
        plt.imshow(self.frame)
        plt.ylim(0, self.size[1])
        plt.xlim(0, self.size[0])
        c2 = Circle(Point(three_points[0]), Point(three_points[1]), Point(three_points[2]))
        self.radius = float(c2.radius)
        self.xcenter = float(c2.center.x)
        self.ycenter = float(c2.center.y)
        c = patches.Circle((self.xcenter, self.ycenter), self.radius, fill=False, color="green", linewidth=2)
        fig1.add_artist(c)
        plt.scatter(self.xcenter, self.ycenter, c = "green", s = 50)
        plt.scatter(three_points[2][0], three_points[2][1], s=100, c ="yellow")
        plt.scatter(three_points[1][0], three_points[1][1], s=100, c ="yellow")
        plt.scatter(three_points[0][0], three_points[0][1], s=100, c ="yellow")
        plt.title('current file: {}'.format(self.filename))


        fig.add_subplot(gs[2, 0])
        plt.imshow(self.frame)
        plt.scatter(three_points[0][0], three_points[0][1], s=100, c ="yellow")
        plt.xlim(three_points[0][0]-25, three_points[0][0]+25)
        plt.ylim(three_points[0][1]-25, three_points[0][1]+25)
        plt.title('Point 1')

        fig.add_subplot(gs[2, 1])
        plt.imshow(self.frame)
        plt.scatter(three_points[1][0], three_points[1][1], s=100, c ="yellow")
        plt.xlim(three_points[1][0]-25, three_points[1][0]+25)
        plt.ylim(three_points[1][1]-25, three_points[1][1]+25)
        plt.title('Point 2')

        fig.add_subplot(gs[2, 2])
        plt.imshow(self.frame)
        plt.scatter(three_points[2][0], three_points[2][1], s=100, c ="yellow")
        plt.xlim(three_points[2][0]-25, three_points[2][0]+25)
        plt.ylim(three_points[2][1]-25, three_points[2][1]+25)
        plt.title('Point 3')
        
        plt.show()
        
        
    def defish(self):
        img_out = ""

        xcenter = int(self.xcenter)+1 #round up
        ycenter = int(self.ycenter)+1
        radius = int(self.radius)+1
        space = 300
        frame_y_start = ycenter+space-radius
        frame_y_end = ycenter+space+radius
        frame_x_start = xcenter+space-radius
        frame_x_end = xcenter+space+radius

        video = imageio.get_reader(self.video_path,  'ffmpeg')
        defished_movie = None

        for frame_index in range(self.frame_number):
            frame = video.get_data(frame_index)
            padded_single_color_frames = []
            for rgb_index in range(3):
                frame_single_color_channel = frame[..., rgb_index].copy()
                frame_single_color_channel = np.pad(frame_single_color_channel, space, constant_values = 255)
                padded_single_color_frames.append(frame_single_color_channel)

            padded_frame = np.asarray(padded_single_color_frames)
            padded_frame = np.moveaxis(padded_frame, 0, -1)
            padded_frame = padded_frame[frame_y_start:frame_y_end, frame_x_start:frame_x_end] #crop the image
            obj = Defisheye(padded_frame, dtype=defish_dtype, format=defish_format, fov=defish_fov, pfov=defish_pfov, radius=radius, xcenter=radius, ycenter=radius)
            defish_frame = obj.convert(img_out)        

            if type(defished_movie) == type(None):
                defished_movie = np.expand_dims(defish_frame, 0)
            else:
                defish_frame_in_4d = np.expand_dims(defish_frame, 0)
                defished_movie = np.append(defished_movie, defish_frame_in_4d, axis=0)

            if frame_index%10 == 0:
                print(frame_index)


        writer = imageio.get_writer(self.video_output_path, fps=self.fps)

        for frame_index in range(defished_movie.shape[0]):
            writer.append_data(defished_movie[frame_index])
        writer.close()
    

In [10]:
class circle_gui(video):
    def __init__(self, l_videos):
        self.l_videos = l_videos
        self.circle_idx = 0
        self.create_gui()
        
    def create_gui(self):
        load_next_button = widgets.Button(description="Load next", style = {'description_width': 'auto'})
        load_next_button.on_click(self.on_load_next_button_click)
        interactive_widget = widgets.interactive(self.l_videos[self.circle_idx].f, Point1_x = widgets.IntText(value = 70, description='x Point1'), Point1_y = widgets.IntText(value = 400, description='y Point1'), Point2_x = widgets.IntText(value = 200, description='x Point2'),Point2_y = widgets.IntText(value = 100, description='y Point2'),Point3_x = widgets.IntText(value = 1120, description='x Point3'),Point3_y = widgets.IntText(value = 50, description='y Point3'))
        row0 = HBox(interactive_widget.children[0:2])
        row1 = HBox(interactive_widget.children[2:4])
        row2 = HBox(interactive_widget.children[4:6])
        row3 = HBox([interactive_widget.children[-1]])
        box = VBox([row0, row1, row2, row3, load_next_button])
        display(box)
        
    def on_load_next_button_click(self, b):
        self.circle_idx+=1

In [11]:
PATH = "C:\\Users\\kobel\\Documents\\Medizin\\Doktorarbeit\\Data\\Crop_Videos\\"

In [12]:
main(PATH)